# Predicción para Kaggle

Incluye:

- Modelo optimizado, semilla única
- Semillerío
- Semillerío con modelo Denicolay

# Predicción para Kaggle

Incluye:

- Modelo optimizado, semilla única
- Semillerío
- Semillerío con modelo Denicolay

In [ ]:
# Librerias
import os
import pandas as pd
import numpy as np
import datetime
import joblib

####################
# funciones y clases
from utils import psi, drift_deflacion

#######
# rutas
# datasets
from config import dataset_file_fe6_6pqt, dataset_file_fe6_6xpqt
# optimizacion
from config import db_path
# modelos
from config import modelos_path
# predicciones
from config import pred_path

##########
# pipeline
from processing import ModelPipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

##############
# optimización
import optuna

#########
# modelos
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import warnings

# Ignorar advertencias de tipo UserWarning
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')


Variables de train y test

In [2]:
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train_all = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
                 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
                 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106]

mes_train_ult_3_meses = [202104, 202105, 202106]

mes_train_ult_6_meses = [202101, 202102, 202103, 202104, 202105, 202106]

mes_train_ult_9_meses = [202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106]

mes_train_ult_anio = [202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106]

mes_train = [202106]
mes_test = 202108

threshold = 0.025

semillas = [437809, 327347, 392879, 455783, 217163]

## Loading data

In [ ]:
data = pd.read_parquet(dataset_file_fe6_6xpqt)

label_mapping = {'CONTINUA': 0, 'BAJA+1': 1, 'BAJA+2': 2}

data['clase_ternaria'] = data['clase_ternaria'].map(label_mapping)

# train data con 9 meses
meses_train = 9
X_train = data[data['foto_mes'].isin(mes_train_ult_9_meses)]
y_train = X_train['clase_ternaria']
X_train = X_train.drop(columns=['clase_ternaria'])

X_kaggle = data[data['foto_mes'] == mes_test]
X_kaggle = X_kaggle.drop(columns=['clase_ternaria']) # nulls

del data

Preprocesando data

In [ ]:
# Imputacion de Xs
cols_with_all_nan = X_train.columns[X_train.isna().all()].tolist()
print("Columns with all NaN values:", cols_with_all_nan)
X_train = X_train.drop(columns=cols_with_all_nan)
X_kaggle = X_kaggle.drop(columns=cols_with_all_nan)

# Imputación de nulls
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_imp = pd.DataFrame(imp_median.fit_transform(X_train), columns=X_train.columns)
X_kaggle_imp = pd.DataFrame(imp_median.transform(X_kaggle), columns=X_train.columns)

del X_train
del X_kaggle

# Codificar variables categóricas
categorical_features = [col for col in X_train_imp.columns if X_train_imp[col].dtype == 'object']

# Convertir variables categóricas a 'category' dtype para LightGBM
for col in categorical_features:
    X_train_imp[col] = X_train_imp[col].astype('category')
    X_kaggle_imp[col] = X_kaggle_imp[col].astype('category')

## Modelo

**Prepro in 6 months and Conceptual FE 6 months**

> comp02_prepro_6.ipynb

> comp02_fe6_6.ipynb

**Usando los últimos 3 meses para optimizar de forma local**

Seleccionado según:

> comp02_pipeline_comp.ipynb

In [ ]:
# Condiciones de la optimización
s = 1
prepro = 6 # data quality + data drifting
fe = 6 # feature engineering conceptual 6 meses
opt = 3 # 3 meses de optimización

# Definir el almacenamiento de Optuna
storage_name = "sqlite:///" + db_path + "optimization_tree.db"

# carga local
# storage_name = "sqlite:///optimizacion/optimization_tree.db"
study_name = f"exp_xgb_pr{prepro}_fe{fe}_tr{opt}_x"

study = optuna.load_study(study_name = study_name,
                                 storage = storage_name)

# Mejores parámetros
opt_params = study.best_trial.params
opt_params.update({'n_jobs': -1})

# Entrenamiento
model = XGBClassifier(**opt_params)
print("Entrenando modelo con:")
print(opt_params)
model.fit(X_train_imp, y_train)

Guardando modelo

In [ ]:
ft = "%dT-%m-%Y%H-%M-%S"
t_now = datetime.datetime.now().strftime(ft)

model_name = f"xgb_pr{prepro}_fe{fe}x_op{opt}_tr{meses_train}_"+t_now

model_path = modelos_path + model_name +".pkl"

# Save the model
joblib.dump(model, model_path)

## Predicción simple

In [ ]:
# model = joblib.load(model_path)

# carga local
# model = joblib.load('modelos/modelos_xgb_pr6_fe6_op3_tr9_22T-11-202409-31-43.pkl')

numero_de_cliente = X_kaggle_imp['numero_de_cliente'].astype(int)

# Prediccion
y_pred_proba = model.predict_proba(X_kaggle_imp)
proba_baja2 = y_pred_proba[:,2] # Según encoding mapping

df_proba = pd.DataFrame({
    'numero_de_cliente': numero_de_cliente.values,
    'proba_baja2': proba_baja2
})

# Umbral
thr_opt_mean = 0.0156
thr_opt_median = 0.0148
prediccion = (proba_baja2 >= thr_opt_median).astype(int).tolist()

Preparando entrega

In [ ]:
submission = pd.DataFrame({
    'numero_de_cliente': numero_de_cliente.values,
    'Predicted': prediccion
})

In [ ]:
# Imprimir value counts de las predicciones
value_counts = submission['Predicted'].value_counts()
total_count = len(submission)
print("\nValue Counts:")
print(value_counts)
print("\nFrecuencia Relativa:")
print((value_counts / total_count) * 100)

submission.info()

Guardando entrega simple y probas

In [ ]:
ft = "%dT-%m-%Y%H-%M-%S"
t_now = datetime.datetime.now().strftime(ft)

pred_name = model_name+".csv"

pred_file = pred_path + pred_name
proba_file = pred_path + "probas/" + pred_name

# Guardar el DataFrame en un archivo CSV
submission.to_csv(pred_file, index=False)
print(f"Predicciones guardadas en {pred_file}")
# Guardar el DataFrame en un archivo CSV
submission.to_csv('predicciones/'+pred_name, index=False)
print(f"Y en {'predicciones/'+pred_name}")

# Guardamos las probas
df_proba.to_csv(proba_file, index=False)
print(f"Probas guardadas en {proba_file}")
df_proba.to_csv('predicciones/probas/'+pred_name, index=False)
print(f"Y en {'predicciones/probas/'+pred_name}")

## Semillerío con modelo Denicolay

Datos X, modificado para compenzar la falta de variables

In [ ]:
semillero_params = {'n_estimators': 23,
                  'num_leaves': 32,
                  'learning_rate': 0.34,
                  'min_data_in_leaf': 711,
                  'feature_fraction': 2*0.2, # x2 para tratar de compenzar la falta de variables
                  'extra_trees': False,
}
semillero_params.update({'n_jobs': -1})

print("Running semillerío for Semillerio Denicolay")
print(semillero_params)

# para registrar las probabilidades
df_sem_proba = pd.DataFrame({
                            'client': numero_de_cliente.values,
                        })

j = 0
s_total = 100
for s in range(217163, 455783, 7*7*7*7): # con 100 semillas
    # nueva instancia del modelos con semilla
    seed = s+(7+j)
    model = LGBMClassifier(**semillero_params, random_state=seed)
    # entreno
    print(f"Entrenando modelo con semilla: {seed}, {j+1} de {s_total}")
    model.fit(X_train_imp, y_train)
    # predigo proba
    y_pred_proba = model.predict_proba(X_kaggle_imp)
    # proba baja+2
    proba_baja2 = y_pred_proba[:,2]
    df_sem_proba[f'proba_s{seed}'] = proba_baja2
    j += 1

# Promediando proba de cada semilla
proba_s_columns = df_sem_proba.filter(regex='^proba_s')
proba_s_mean = proba_s_columns.mean(axis=1)

df_sem_proba['proba_sem_mean'] = proba_s_mean

# Umbral
thr_opt_sem = 0.016 # # segun comp02_back-testing con semx

# Prediccion
df_sem_proba['pred'] = np.where(df_sem_proba.proba_sem_mean >= thr_opt_sem, 1, 0)

df_sem_proba.head()

Preparando entrega

In [ ]:
submission = pd.DataFrame({
    'numero_de_cliente': numero_de_cliente.values,
    'Predicted': df_sem_proba['pred'].values
})

# Imprimir value counts de las predicciones
value_counts = submission['Predicted'].value_counts()
total_count = len(submission)
print("\nValue Counts:")
print(value_counts)
print("\nFrecuencia Relativa:")
print((value_counts / total_count) * 100)

submission.info()

Guardando pred semillerio denicolay

In [ ]:
ft = "%dT-%m-%Y%H-%M-%S"
t_now = datetime.datetime.now().strftime(ft)

pred_name = f"lgbm_mod_pr{prepro}_fe{fe}x_tr{meses_train}_sem{s_total}_"+t_now+".csv"

pred_file = pred_path + pred_name
proba_file = pred_path + "probas/" + pred_name

# Guardar el DataFrame en un archivo CSV
submission.to_csv(pred_file, index=False)
print(f"Predicciones guardadas en {pred_file}")
# Guardar el DataFrame en un archivo CSV
submission.to_csv('predicciones/'+pred_name, index=False)
print(f"Y en {'predicciones/'+pred_name}")

# Guardamos las probas
df_sem_proba.to_csv(proba_file, index=False)
print(f"Probas guardadas en {proba_file}")
df_sem_proba.to_csv('predicciones/probas/'+pred_name, index=False)
print(f"Y en {'predicciones/probas/'+pred_name}")

## Semillerío con modelo Denicolay

Datos X

In [ ]:
semillero_params = {'n_estimators': 23,
                  'num_leaves': 32,
                  'learning_rate': 0.34,
                  'min_data_in_leaf': 711,
                  'feature_fraction': 2*0.2, # x2 para tratar de compenzar la falta de variables
                  'extra_trees': False,
}
semillero_params.update({'n_jobs': -1})

print("Running semillerío for Semillerio Denicolay")
print(semillero_params)

# para registrar las probabilidades
df_sem_proba = pd.DataFrame({
                            'client': numero_de_cliente.values,
                        })

j = 0
s_total = 100
for s in range(217163, 455783, 7*7*7*7): # con 100 semillas
    # nueva instancia del modelos con semilla
    seed = s+(7+j)
    model = LGBMClassifier(**semillero_params, random_state=seed)
    # entreno
    print(f"Entrenando modelo con semilla: {seed}, {j+1} de {s_total}")
    model.fit(X_train_imp, y_train)
    # predigo proba
    y_pred_proba = model.predict_proba(X_kaggle_imp)
    # proba baja+2
    proba_baja2 = y_pred_proba[:,2]
    df_sem_proba[f'proba_s{seed}'] = proba_baja2
    j += 1

# Promediando proba de cada semilla
proba_s_columns = df_sem_proba.filter(regex='^proba_s')
proba_s_mean = proba_s_columns.mean(axis=1)

df_sem_proba['proba_sem_mean'] = proba_s_mean

# Umbral
thr_opt_sem = 0.013 # # segun comp02_back-testing con semx

# Prediccion
df_sem_proba['pred'] = np.where(df_sem_proba.proba_sem_mean >= thr_opt_sem, 1, 0)

df_sem_proba.head()

Preparando entrega

In [ ]:
submission = pd.DataFrame({
    'numero_de_cliente': numero_de_cliente.values,
    'Predicted': df_sem_proba['pred'].values
})

# Imprimir value counts de las predicciones
value_counts = submission['Predicted'].value_counts()
total_count = len(submission)
print("\nValue Counts:")
print(value_counts)
print("\nFrecuencia Relativa:")
print((value_counts / total_count) * 100)

submission.info()

Guardando pred semillerio denicolay

In [ ]:
ft = "%dT-%m-%Y%H-%M-%S"
t_now = datetime.datetime.now().strftime(ft)

pred_name = f"lgbm_pr{prepro}_fe{fe}x_tr{meses_train}_sem{s_total}_"+t_now+".csv"

pred_file = pred_path + pred_name
proba_file = pred_path + "probas/" + pred_name

# Guardar el DataFrame en un archivo CSV
submission.to_csv(pred_file, index=False)
print(f"Predicciones guardadas en {pred_file}")
# Guardar el DataFrame en un archivo CSV
submission.to_csv('predicciones/'+pred_name, index=False)
print(f"Y en {'predicciones/'+pred_name}")

# Guardamos las probas
df_sem_proba.to_csv(proba_file, index=False)
print(f"Probas guardadas en {proba_file}")
df_sem_proba.to_csv('predicciones/probas/'+pred_name, index=False)
print(f"Y en {'predicciones/probas/'+pred_name}")